### Automatic English class test creator

This notebook automatically generates English class test using examples from site: https://www.gettinenglish.com

Students names are read from file: uczniowie.txt

Test are different for every student and are saved in separate folder with names in files.

Separate files with answers are generated.

(Should be customizable, changing webpage should only demand finding proper tag on webpage.)

In [1]:
import random, os, bs4, requests
from string import Template

In [2]:
def delete_polish_signs(text):
    d = {"ą":"a","ę":"e","ł":"l","ś":"s","ć":"c","ż":"z","ź":"z","ń":"n","ó":"o"}
    return "".join([d[c] if c in d.keys() else c for c in text])

In [3]:
def get_tasks(urls, tag, task_out_sep, task_in_sep, reverse = False):
    """
    Parser do url-i.
    Pobieramy adresy URL oraz tagi i zapisujemu polecenia do słownika:
        key: pytanie
        value: odpowiedź
    Proces jest ujednolicony przez co trochę mało optymalnie pewne rzeczy są.
    Najpierw szukamy kontenerów z przykładami.
    Rozdzielamy kontener na przypadki
    Rozdizelamy na pytanie i odpowiedź
    
    Takie działanie sprawdza się w przypadku strony z zadaniami z angielskiego, na której testuje.
    Na innych stronach ciężej pewnie będzię z określeniem separatorów jeśli struktura będzie inna.
    """
    result = {}
    if type(urls) != list:
        urls = [urls]
    for url in urls:
        req = requests.get(url, headers={'User-Agent': 'Chrome'})
        req.encoding = 'utf-8'
        req.raise_for_status()
        soup = bs4.BeautifulSoup(req.text,'html.parser')
        
        temp = soup.body
        temp = temp.find_all(attrs = {tag[0]:tag[1]})
        for task_pack in temp:
            tasks = [task_pack]
            if '<' not in task_out_sep:
                tasks = task_pack.get_text().split(task_out_sep)
            else:
                tasks = task_pack.find_all(task_out_sep.replace('<','').replace('>',''))
                tasks = [task.get_text() for task in tasks]
                
            for task in tasks:
                try:
                    eng, pol = task.split(task_in_sep)
                    if reverse:
                        result[eng] = pol
                    else:
                        result[pol] = eng
                except:
                    pass
    return result

In [4]:
def random_tasks(n, dic, desc, sep):
    """
    Funkcja generuje tekst skryptu LateX-owego w formacie
    Opis polecenia
        -losowy podpunkt
        -losowy podpunkt
        -...
    Generuje zarówno polecenie, jak i polecenie z odpowiedzią.
    W zwizku z tym że i tak mamy tu dostęp do obu informacji.
    """
    if sep =='\n':
        sep = r'\\'
    else:
        sep = r'\quad'+sep+r'\quad'
    tasks = []
    anwsers = []
    for i in range(n):
        word = random.choice(list(dic.keys()))
        tasks.append("\item " + word + sep)
        anwsers.append("\item " + word + sep + "{\color{red}" + dic[word]+'}')
    
    tasks = '\n'.join(f'{z}' for i, z in enumerate(tasks, 1))
    anwsers = '\n'.join(f'{z}' for i, z in enumerate(anwsers, 1))
    
    tasks = desc + '\n\\begin{enumerate}\n' + tasks + '\n\\end{enumerate}\n'
    anwsers = desc + '\n\\begin{enumerate}\n' + anwsers + '\n\\end{enumerate}\n'
    return tasks, anwsers

In [5]:
exam_template = r'''
\documentclass{article}
\usepackage[T1]{fontenc}
\usepackage[utf8]{inputenc}
\usepackage{lmodern}
\usepackage{textcomp}
\usepackage{lastpage}
\usepackage{amsmath}
\usepackage{xcolor}

\begin{document}
\normalsize
${STUDENT}
\section*{Kartkówka Angielski}
${ZADANIE}
Rozwiązania odeślij na mail: anowak@ok.pl do godziny 16:00.
A. Nowak
\end{document}
'''

In [6]:
def create_exam(folder_name, file_name, students_list_path, exam_template, task_list):
    """
    Function generates final LaTex files.
    It generates test and anwser files in separate path.
    
    :param str folder_name: folder name to save the result in
    :param str file_name: core file name for results
    :param str student_list_path: path to students list
    :param raw str exam_template: LaTex template, to avoid errors raw strings are recommended
    :param list of dict task_list:
    """
    tt = Template(exam_template)
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    tasks_bases = []
    for task in task_list:
        tasks_bases.append(get_tasks(task['urls'], task['tag'], task['out_sep'], task['in_sep'], task['reverse']))

    with open(students_list_path,"r") as file:
        names = file.readlines()
        for name in names:
            imie, nazwisko = name.split(" ")
            imie, nazwisko = delete_polish_signs(imie.strip()),delete_polish_signs(nazwisko.strip())
            with open(folder_name+"/"+file_name+"_"+nazwisko+"_"+imie+".tex","w") as outfile,\
                open(folder_name+"/"+file_name+"_"+nazwisko+"_"+imie+"_odp.tex","w") as anwserfile:
                zads = ''
                odps = ''
                for i in range(len(task_list)):
                    zad, odp = random_tasks(5, tasks_bases[i], task_list[i]['desc'], sep = task_list[i]['in_sep'])
                    zads += zad
                    odps += odp
                
                outtext = tt.substitute(STUDENT = nazwisko +" "+ imie, ZADANIE = zads)
                outfile.write(outtext)
                
                outtext = tt.substitute(STUDENT = nazwisko +" "+ imie, ZADANIE = odps)
                anwserfile.write(outtext)

In [7]:
########################
# Parametry które można zmienić
########################
strony_ze_slowami=['https://www.gettinenglish.com/angielskie-slownictwo-z-prasy/',
                    'https://www.gettinenglish.com/sporty-zimowe-angielskie-slownictwo/',
                    'https://www.gettinenglish.com/angielskie-wyrazenia-zwiazane-z-pieniedzmi/',
                    'https://www.gettinenglish.com/angielskie-slownictwo-zwiazane-z-terroryzmem/']

strony_ze_zdaniami = ['https://www.gettinenglish.com/present-simple/',
                     'https://www.gettinenglish.com/present-continuous/',
                     'https://www.gettinenglish.com/present-perfect/',
                     'https://www.gettinenglish.com/present-perfect-continuous/',
                     'https://www.gettinenglish.com/past-simple-2/',
                     'https://www.gettinenglish.com/past-continuous/',
                     'https://www.gettinenglish.com/past-perfect/',
                     'https://www.gettinenglish.com/past-perfect-continuous/',
                     'https://www.gettinenglish.com/future-simple/',
                     'https://www.gettinenglish.com/future-continuous/',
                     'https://www.gettinenglish.com/future-perfect/',
                     'https://www.gettinenglish.com/future-perfect-continuous/',
                     'https://www.gettinenglish.com/be-going-to/']

# Najważnieszy parametr, musimy dla każdego zadania sprecyzować parametry.
# Robiłem to z myślą o stronie gettinenglish.
# Na niej artykuły pisane są w ten sposób że pare przykładów zapisanych jest pod jednym tagiem

# desc    - polecenie które jest niezmienne dla każdego ucznia
# urls    - ulr, bądź lista url-i skąd mamy pobrać treści podpunktów(muszą być tej samej sruktury, odnosić się do tych samych tagów)
# tag     - tag który pozwala znaleź kontener z przykładami
# out_sep - separator przykładów wewnątrz kontenera(dla słów jest to \n, dla zdań każdy jest w osobnym tagu listowym <li>
#           ale żeby było bardziej uniwersalnie to podaje że separatorem jest <li>, w kodzie uzywam wtedy find_all drugi raz)
# in_sep  - separator części napisanej po polsku, a części angielskiej
# reverse - Czy ma być tłumaczenie z polskiego na angielski czy na odwrót
task_list = [
    {'desc':'Zad.1. Przetłumacz następujące słowa na polski',
     'urls':strony_ze_slowami,
     'tag':('class','col-xs-6 bs-shortcode-col'),
     'out_sep':'\n',
     'in_sep':'–',
     'reverse': False},
    {'desc':'Zad.2. Przetłumacz następujące słowa na angielski',
     'urls':strony_ze_slowami,
     'tag':('class','col-xs-6 bs-shortcode-col'),
     'out_sep':'\n',
     'in_sep':'–',
     'reverse': True},
    {'desc':'Zad.3. Przetłumacz zdania na angielski',
     'urls':strony_ze_zdaniami,
     'tag':('class','bs-shortcode-list list-style-edit'),
     'out_sep':'<li>',
     'in_sep':'\n',
     'reverse': False}
]


In [8]:
create_exam('test','test_nr_1_english','data/uczniowie.txt',exam_template,task_list)

### Compiling LaTex files and deleting leftover files

In [9]:
%%bash
cd test
for i in *.tex; do pdflatex $i; done
rm *.aux
rm *.tex
rm *.log
cd ..

This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./test_nr_1_english_Dabrowski_Stanislaw_odp.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/fontenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texmf/tex/latex/lm/lmodern.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/textcomp.sty)
(/usr/share/texlive/texmf-dist/tex/latex/lastpage/lastpage.sty)
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsmath.sty
For additional information on amsmath, use the `?' option.
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amstext.sty
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsgen.sty))
(/usr/share/t